In [1]:
import numpy as np
import pandas as pd
import glob
import ipyparallel as p
import os
import ntpath
import ProgressBar as pb
from pymail import alert
import time

In [2]:
exname = 'smex7_sen'
method = '7'
site = 'P301'

In [3]:
files = glob.glob('/Volumes/data/RHESSys_out/P301/%s/*_basin.daily'%exname)

In [4]:
len(files)

250000

In [5]:
params = pd.read_pickle('./data/soil_params_multispinup_%s_smex%s_sen.pcl'%(site,method))

In [33]:
params.combo.unique()

array(['d1:s1', 'd1:s2', 'd1:s3', 'd1:s4', 'd1:s5', 'd2:s1', 'd2:s2',
       'd2:s3', 'd2:s4', 'd2:s5', 'd3:s1', 'd3:s2', 'd3:s3', 'd3:s4',
       'd3:s5', 'd4:s1', 'd4:s2', 'd4:s3', 'd4:s4', 'd4:s5', 'd5:s1',
       'd5:s2', 'd5:s3', 'd5:s4', 'd5:s5'], dtype=object)

In [29]:
params.head()

,MAP,PeakSWE,SMR,TpeakSWE,combo,idx,ksat,m,pa,po,...,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate
0,0.325,0.521211,0.014403,113.0,d1:s1,0,757.89,1.2548,0.050794,0.516732,...,3273.973193,0.0,1097.506537,0.000000,4930.146548,11333.670226,72.746216,392564.529297,0.0,0.418232
1,0.325,0.521211,0.014403,113.0,d1:s2,0,757.89,1.2548,0.076191,0.516732,...,3124.028192,0.0,1026.603452,73.020907,4854.259079,11159.216274,72.746216,392564.603777,0.0,0.418232
2,0.325,0.521211,0.014403,113.0,d1:s3,0,757.89,1.2548,0.101588,0.516732,...,2978.818824,0.0,969.485761,120.443883,4798.196733,11030.337318,72.746216,392564.603777,0.0,0.418232
3,0.325,0.521211,0.014403,113.0,d1:s4,0,757.89,1.2548,0.126985,0.516732,...,2826.330069,0.0,1055.101881,132.551484,4873.304786,11202.999509,72.746216,392564.603777,0.0,0.418232
4,0.325,0.521211,0.014403,113.0,d1:s5,0,757.89,1.2548,0.152382,0.516732,...,2683.178636,0.0,918.802707,149.791240,4748.277188,10915.579744,72.746216,392564.603777,0.0,0.418232


In [6]:
files = pd.DataFrame({'basin_daily':files})

In [7]:
def getIDX(fl):
    return int(fl.split('_')[-2])

In [8]:
files['scenidx'] = files.basin_daily.map(getIDX)

In [9]:
params = pd.merge(params,files, on= 'scenidx')

In [10]:
from rhessys import snowmelt_experiment as smex

In [11]:
params['Tnosnow'] = params.TpeakSWE+np.ceil(params.PeakSWE/params.SMR)

In [12]:
params['date_peakSWE'] = params.apply(smex.DOPEAKSWE2cal,axis=1)
params['date_ONS'] =params.apply(smex.DONS2cal,axis=1)

In [13]:
c = p.Client()
view = c.load_balanced_view()

In [14]:
c.ids

[0, 1, 2, 3, 4, 5, 6]

In [15]:
len(params)

250000

In [16]:
dr = pd.date_range('2008-10-1','2009-10-02',freq='D')

In [17]:
%px dr = pd.date_range('2008-10-1','2009-10-02',freq='D')

In [18]:
def processSM(bd,idx,strt,nd):

    # read in the basin daily data
    data = pd.read_csv(bd,delimiter=' ')

    data.index = pd.DatetimeIndex(dr)

    data['ET'] = data.evap + data.trans
    trans = data.trans.sum()
    evap = data.evap.sum()
    ET = data.ET.sum() # compute total ET
    Q = data.streamflow.sum() # compute total streamflow 
    Qbf = data.baseflow.sum() # compute total baseflow
    Qrf = data['return'].sum() # compute total returnflow,p
    sm = data.snowmelt.sum()
    rech = data.recharge.sum() # compute total recharge
    sat = data['%sat_area'].mean() # compute the mean saturated area
    p = data.precip.sum() # compute water year precip
    rz_storage = data.rz_storage.mean() # root zone storage
    rz_drainage = data.rz_drainage.mean() # root zone drainage 
    unsat_drainage = data.unsat_drain.mean() # unsaturated zone drainage
    unsat_storage = data.unsat_stor.mean() # unsaturated zone drainage
    sat_def = data.sat_def.mean() # saturation deficite (mm)
    sat_def_z = data.sat_def_z.mean() # saturation deficite depth, is this the depth to ground water?
    gwStore = data['gw.storage'].sum() # gw store
    gwFlux = data['gw.Qout'].mean()
    ETRate = data.ET.mean()
    
    # create full output object
    full = (idx,trans,evap,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,
            unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate)
    
    data = data.loc[strt:nd] # crop the data to the melt season
    T = data.trans.sum() # compute totoal transpiration
    E = data.evap.sum() # compute total evaporation
    ET = data.ET.sum() # compute total ET
    Q = data.streamflow.sum() # compute total streamflow 
    Qbf = data.baseflow.sum() # compute total baseflow
    Qrf = data['return'].sum() # compute total returnflow,p
    sm = data.snowmelt.sum()
    rech = data.recharge.sum() # compute total recharge
    sat = data['%sat_area'].mean() # compute the mean saturated area
    p = data.precip.sum() # compute water year precip
    rz_storage = data.rz_storage.mean() # root zone storage
    rz_drainage = data.rz_drainage.mean() # root zone drainage 
    unsat_drainage = data.unsat_drain.mean() # unsaturated zone drainage
    unsat_storage = data.unsat_stor.mean() # unsaturated zone drainage
    sat_def = data.sat_def.mean() # saturation deficite (mm)
    sat_def_z = data.sat_def_z.mean() # saturation deficite depth, is this the depth to ground water?
    gwStore = data['gw.storage'].sum() # gw store
    gwFlux = data['gw.Qout'].mean()
    ETRate = data.ET.mean()
    
    melt = (idx,T,E,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,
            unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate)
    
    return full,melt

In [19]:
idx = 0
print params.basin_daily[idx]
processSM(params.basin_daily[idx],params.scenidx[idx],params.date_peakSWE[idx],params.date_ONS[idx])

/Volumes/data/RHESSys_out/P301/smex7_sen/smex7_sen_0_basin.daily


((0,
  455.50006999999994,
  104.96986,
  560.46993,
  18.431646,
  18.431646,
  0.0,
  362.16081799999995,
  0.0,
  716.4708499999999,
  3250.571043212535,
  0.018942937329700274,
  1099.9170150381399,
  2.1667303760217984,
  4947.251389275205,
  11372.991699430522,
  397.726705,
  3807342.936078,
  0.0,
  1.5271660217983654),
 (0,
  10.164971000000001,
  5.7278389999999995,
  15.89281,
  2.012645,
  2.012645,
  0.0,
  40.592388,
  0.0,
  0.0,
  3273.973193368419,
  0.0,
  1097.5065370000004,
  0.0,
  4930.146548315789,
  11333.670225921049,
  72.746216,
  392564.5292970001,
  0.0,
  0.41823184210526315))

In [20]:
#res = view.map(processSM,params.basin_daily[0:1000],params.scenidx[0:1000],params.date_peakSWE[0:1000],params.date_ONS[0:1000])
res = view.map(processSM,params.basin_daily,params.scenidx,params.date_peakSWE,params.date_ONS)

In [21]:
prog = pb.ProgressBar(len(params))

In [22]:
while res.ready() == False:
    prog.animate_ipython(res.progress)
    time.sleep(2)

#alert.send_alert('barnhatb@colorado.edu','Processing Nr1 %s has finished'%exname,'Your script has finished')

[****************100%******************]  249947 of 250000 complete                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [23]:
res.done()

True

In [24]:
res.progress

250000

In [25]:
full,melt = zip(*res.result())

## Merge processing results with parameters dataset

In [34]:
idx,trans,evap,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate = zip(*full)

params = pd.read_pickle('./data/soil_params_multispinup_%s_smex%s_sen.pcl'%(site,method))

# make a date frame of the results
proc = pd.DataFrame()

arrs = [idx,trans,evap,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,
        sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate]
keys = ['scenidx','trans','evap','ET','Q','Qbf','Qrf','rech','sat','p','rz_storage','rz_drainage',
        'unsat_storage','unsat_drainage','sat_def','sat_def_z','sm','gwStore','gwFlux','ETRate']

for arr,key in zip(arrs,keys):
    proc[key] = arr # insert the results

params = pd.merge(params,proc,on='scenidx') # join on file index number
params.dropna(inplace=True) # drop missing values

params.to_hdf('./data/%s_%s_full.hdf'%(site,exname),'df',format='fixed',complevel=5,complib='bzip2',fletcher32=True)

In [35]:
params.head()

,MAP,PeakSWE,SMR,TpeakSWE,combo,idx,ksat,m,pa,po,...,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate
0,0.325,0.521211,0.014403,113.0,d1:s1,0,757.89,1.2548,0.050794,0.516732,...,3250.571043,0.018943,1099.917015,2.166730,4947.251389,11372.991699,397.726705,3.807343e+06,0.0,1.527166
1,0.325,0.521211,0.014403,113.0,d1:s2,0,757.89,1.2548,0.076191,0.516732,...,3101.757169,0.000000,1036.515251,46.963793,4880.200172,11218.850969,397.726705,3.807344e+06,0.0,1.527166
2,0.325,0.521211,0.014403,113.0,d1:s3,0,757.89,1.2548,0.101588,0.516732,...,2956.260288,0.000000,1001.767320,89.677185,4847.631834,11143.981228,397.726705,3.807344e+06,0.0,1.527166
3,0.325,0.521211,0.014403,113.0,d1:s4,0,757.89,1.2548,0.126985,0.516732,...,2803.562558,0.000000,976.767720,124.833347,4813.739275,11064.818086,397.726705,3.807344e+06,0.0,1.527166
4,0.325,0.521211,0.014403,113.0,d1:s5,0,757.89,1.2548,0.152382,0.516732,...,2658.737463,0.000000,905.667577,139.002862,4753.894760,10928.493701,397.726705,3.807344e+06,0.0,1.527166


In [36]:
len(params)

250000

In [37]:
idx,T,E,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate = zip(*melt)

params = pd.read_pickle('./data/soil_params_multispinup_%s_smex%s_sen.pcl'%(site,method))

# make a date frame of the results
proc = pd.DataFrame()

arrs = [idx,T,E,ET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,
        sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate]
keys = ['scenidx','T','E','ET','Q','Qbf','Qrf','rech','sat','p','rz_storage','rz_drainage',
        'unsat_storage','unsat_drainage','sat_def','sat_def_z','sm','gwStore','gwFlux','ETRate']

for arr,key in zip(arrs,keys):
    proc[key] = arr # insert the results

params = pd.merge(params,proc,on='scenidx') # join on file index number
params.dropna(inplace=True) # drop missing values

params.to_hdf('./data/%s_%s_melt.hdf'%(site,exname),'df',format='fixed',complevel=5,complib='bzip2',fletcher32=True) # 

In [38]:
params.shape

(250000, 31)

In [28]:
alert.send_alert('barnhatb@colorado.edu','Processing P301 %s has finished'%exname,'Your script has finished')